In [ ]:
# import json
# import re
# import os
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from collections import defaultdict
# from tqdm import tqdm
# import torch

# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction # Added for BLEU

# # --- DATA ---

# def load_menatqa_dataset(file_path='/content/MenatQA.json'):
#     if not os.path.exists(file_path):
#         import urllib.request
#         print("Downloading MenatQA...")
#         urllib.request.urlretrieve(
#             "https://raw.githubusercontent.com/weiyifan1023/MenatQA/main/datasets/MenatQA.json",
#             file_path
#         )
#     with open(file_path, 'r') as f:
#         data = json.load(f)
#     print(f"Loaded MenatQA dataset with {len(data)} examples")
#     return data

# def extract_reasoning_hops(example):
#     question = example.get('question', '')
#     answer = example.get('answer', '')
#     q_type = example.get('type', '')
#     time_scope = example.get('time_scope', '')
#     sentences = [s.strip() for s in question.split('.') if s.strip()]
#     clauses = len([c for c in re.split(r'and|or|but|because|when|if', question.lower()) if c.strip()])
#     capitalized_words = len([w for w in question.split() if w and w[0].isupper()])
#     complexity_score = 1
#     complexity_score += min(1, len(sentences) - 1)
#     complexity_score += min(1, (clauses - 1) // 2)
#     complexity_score += min(1, capitalized_words // 3)
#     if time_scope:
#         complexity_score += 1
#     complexity_score = min(4, max(1, complexity_score))
#     hops = []
#     hops.append(f"Understand the question: {question}")
#     if complexity_score >= 2:
#         if time_scope:
#             hops.append(f"Identify time context: {time_scope}")
#         else:
#             hops.append(f"Recognize question type: {q_type}")
#     if complexity_score >= 3:
#         hops.append("Retrieve relevant facts and information")
#     if complexity_score >= 4:
#         hops.append("Analyze relationships between facts")
#     hops.append(f"Formulate answer: {answer}")
#     return hops, complexity_score

# def preprocess_dataset(data):
#     processed_data = []
#     hop_counts = {1: 0, 2: 0, 3: 0, 4: 0}
#     for item in data:
#         gold_hops, complexity_score = extract_reasoning_hops(item)
#         hop_counts[min(4, complexity_score)] += 1
#         entry = {
#             'ID': item.get('ID', ''),
#             'question': item.get('question', ''),
#             'answer': item.get('answer', ''),
#             'type': item.get('type', ''),
#             'time_scope': item.get('time_scope', ''),
#             'gold_hops': gold_hops,
#             'hop_count': complexity_score,
#             'model_prediction': '',
#             'model_reasoning_steps': []
#         }
#         processed_data.append(entry)
#     df = pd.DataFrame(processed_data)
#     df['hop_category'] = pd.cut(
#         df['hop_count'],
#         bins=[-1, 1, 2, 3, float('inf')],
#         labels=['1-hop', '2-hop', '3-hop', '4+-hop']
#     )
#     print(f"Dataset preprocessed: {len(df)} questions")
#     for i in range(1, 5):
#         print(f"{i}-hop questions: {hop_counts[i]} ({hop_counts[i]/len(df):.1%})")
#     return df

# # --- LLM ---

# def run_qwen_cot_predictions(
#         df,
#         model_name="Qwen/Qwen3-0.6B",
#         device=None,
#         max_new_tokens=256,
#         temperature=0.0):

#     if device is None:
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#     print(f"Loading HuggingFace model {model_name} ({device}) ...")
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto').to(device)
#     pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device == 'cuda' else -1,
#                     max_new_tokens=max_new_tokens, temperature=temperature, do_sample=False)

#     def format_cot_prompt(question):
#         return (f"Answer the following question step by step. "
#                 f"Write each reasoning step on a new line, and finish with your answer.\n"
#                 f"Question: {question}\nLet's think step by step:")

#     all_preds, all_steps = [], []
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating predictions"):
#         prompt = format_cot_prompt(row["question"])
#         try:
#             generation = pipe(prompt)[0]['generated_text']
#         except Exception as e:
#             print(f"Qwen error on idx {idx} (ID: {row.get('ID', 'N/A')}): {e}")
#             generation = ""
#         output = generation[len(prompt):] if generation.startswith(prompt) else generation
#         steps = [x.strip() for x in output.split('\n') if x.strip()]
#         if not steps:
#             steps = [""] # Ensure there's always at least one step (even if empty)
#         model_answer = steps[-1]
#         all_preds.append(model_answer)
#         all_steps.append(steps)
#     df = df.copy()
#     df['model_prediction'] = all_preds
#     df['model_reasoning_steps'] = all_steps
#     return df

# # --- METRICS ---

# def compute_exact_match(prediction, gold):
#     return int(str(prediction).strip().lower() == str(gold).strip().lower())

# def compute_f1(prediction, gold):
#     pred_tokens = str(prediction).lower().split()
#     gold_tokens = str(gold).lower().split()
#     common = set(pred_tokens) & set(gold_tokens)
#     if not pred_tokens or not gold_tokens:
#         return 0.0
#     prec = len(common) / len(pred_tokens)
#     rec = len(common) / len(gold_tokens)
#     if prec + rec == 0:
#         return 0.0
#     f1 = 2 * prec * rec / (prec + rec)
#     return f1

# def compute_bleu(prediction, gold):
#     """
#     Computes sentence BLEU score between prediction and gold answer.
#     """
#     pred_tokens = str(prediction).lower().split()
#     gold_tokens = str(gold).lower().split()

#     if not pred_tokens or not gold_tokens:
#         return 0.0

#     reference = [gold_tokens]
#     candidate = pred_tokens
    
#     weights = (0.25, 0.25, 0.25, 0.25) # BLEU-4
#     chencherry = SmoothingFunction()
#     try:
#         bleu_score = sentence_bleu(reference, candidate, weights=weights, smoothing_function=chencherry.method1)
#     except ZeroDivisionError:
#         bleu_score = 0.0
#     except ValueError: # Handles cases like candidate/reference too short
#         bleu_score = 0.0
#     return bleu_score

# def evaluate_hop_wise_accuracy(df):
#     categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0]) # Sort categories
#     hop_results = {}
#     overall_em, overall_f1, overall_bleu, overall_count = 0, 0, 0, 0
    
#     for cat in categories:
#         subdf = df[df['hop_category'] == cat]
#         if subdf.empty:
#             hop_results[str(cat)] = {'count': 0, 'EM': 0, 'F1': 0, 'BLEU': 0}
#             continue

#         ems = [compute_exact_match(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
#         f1s = [compute_f1(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
#         bleus = [compute_bleu(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
        
#         count = len(subdf)
#         mean_em = np.mean(ems) if ems else 0
#         mean_f1 = np.mean(f1s) if f1s else 0
#         mean_bleu = np.mean(bleus) if bleus else 0
        
#         hop_results[str(cat)] = {'count': count, 'EM': mean_em, 'F1': mean_f1, 'BLEU': mean_bleu}
        
#         overall_em += sum(ems)
#         overall_f1 += sum(f1s)
#         overall_bleu += sum(bleus)
#         overall_count += count
        
#     hop_results['overall'] = {
#         'count': overall_count,
#         'EM': overall_em / overall_count if overall_count else 0,
#         'F1': overall_f1 / overall_count if overall_count else 0,
#         'BLEU': overall_bleu / overall_count if overall_count else 0,
#     }
#     return hop_results

# def evaluate_step_verification(df: pd.DataFrame) -> dict:
#     all_step_matches = []
#     complete_path_matches = 0
#     partial_path_matches = 0
#     total_questions = 0
#     # Ensure categories are sorted for consistent output
#     sorted_categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
#     hop_category_results = defaultdict(lambda: {'matches': 0, 'partial_matches': 0, 'total': 0})

#     for _, row in df.iterrows():
#         gold_hops = row['gold_hops']
#         model_steps = row['model_reasoning_steps']
        
#         if not gold_hops or not model_steps or not model_steps[0]: # Check if model_steps[0] is not empty
#             continue
            
#         total_questions += 1
#         hop_cat = row['hop_category']
#         if hop_cat is None: continue # Skip if hop_category is somehow None

#             hop_category_results[hop_cat]['total'] += 1            
#             step_matches = []
#             first_match = 0
#             last_match = 0
#             middle_matches = []

#         # First step (question understanding)
#         gold_q_terms = set(re.findall(r'\w+', gold_hops[0].lower()))
#         model_q_terms = set(re.findall(r'\w+', model_steps[0].lower())) # model_steps[0] might be empty if parsing failed
#         if gold_q_terms: # Avoid division by zero if gold_q_terms is empty (shouldn't happen with current logic)
#             common_terms = gold_q_terms & model_q_terms
#             first_match = len(common_terms) / len(gold_q_terms)
#         step_matches.append(first_match)

#         # Last step (answer formulation)
#         if len(gold_hops) > 1 and len(model_steps) > 1:
#             gold_ans_text = row['answer'].lower() # The actual gold answer
#             # Gold last hop usually contains the answer
#             # Model last hop is considered the model's answer
#             model_last_hop_text = model_steps[-1].lower()
            
#             # Check if the gold answer text is present in the model's final step (which is its answer)
#             last_match = 1.0 if gold_ans_text in model_last_hop_text else 0.0
#             step_matches.append(last_match)

#         # Intermediate steps
#         if len(gold_hops) > 2 and len(model_steps) > 2: # At least one intermediate step in both
#             # Compare internal reasoning steps (excluding first and last)
#             # Number of intermediate steps to compare is min of available intermediate steps
#             num_intermediate_to_compare = min(len(gold_hops) - 2, len(model_steps) - 2)
#             for i in range(num_intermediate_to_compare):
#                 gold_mid_idx = i + 1
#                 model_mid_idx = i + 1
                
#                 gold_mid = gold_hops[gold_mid_idx].lower()
#                 model_mid = model_steps[model_mid_idx].lower()
                
#                 gold_words = set(re.findall(r'\w+', gold_mid))
#                 model_words = set(re.findall(r'\w+', model_mid))
                
#                 if gold_words and model_words:
#                     common_words = gold_words & model_words
#                     similarity = len(common_words) / max(len(gold_words), len(model_words)) # Jaccard-like on words
#                     middle_matches.append(similarity)
#                     step_matches.append(similarity)
#                 elif gold_words or model_words: # One is empty, the other not
#                     middle_matches.append(0.0)
#                     step_matches.append(0.0)
#                 # If both are empty, it could be a 0/0 case, or simply 0 similarity if one had content.
#                 # The logic above handles if one is empty. If both are empty (e.g. just "."), similarity is low/0.

#         all_step_matches.extend(step_matches)
        
#         if step_matches: # Only if we actually made comparisons
#             # Define complete match: all compared steps must be >= 0.6
#             is_complete_match = all(match >= 0.6 for match in step_matches)

#             # Define partial match (if not a complete match)
#             # First and last steps are critical
#             is_partial_match = False
#             if not is_complete_match:
#                 avg_middle_match = sum(middle_matches) / len(middle_matches) if middle_matches else 1.0 # If no middle, condition passes
#                 if first_match >= 0.7 and last_match >= 0.7 and avg_middle_match >= 0.4:
#                     is_partial_match = True
            
#             if is_complete_match:
#                 complete_path_matches += 1
#                 hop_category_results[hop_cat]['matches'] += 1
#             elif is_partial_match:
#                 partial_path_matches += 1
#                 hop_category_results[hop_cat]['partial_matches'] += 1

#     cot_em_overall = complete_path_matches / total_questions if total_questions > 0 else 0
#     cot_partial_overall = (complete_path_matches + partial_path_matches) / total_questions if total_questions > 0 else 0
    
#     cot_em_by_category = {}
#     cot_partial_by_category = {}

#     for hop_cat_key in sorted_categories: # Use sorted categories for results dict
#         data = hop_category_results[hop_cat_key]
#         cat_total = data['total']
#         cot_em = data['matches'] / cat_total if cat_total > 0 else 0
#         cot_partial = (data['matches'] + data['partial_matches']) / cat_total if cat_total > 0 else 0
#         cot_em_by_category[str(hop_cat_key)] = cot_em
#         cot_partial_by_category[str(hop_cat_key)] = cot_partial
        
#     results = {
#         'CoT-EM': cot_em_overall,
#         'CoT-Partial': cot_partial_overall,
#         'Hop-Match-Rate': np.mean(all_step_matches) if all_step_matches else 0,
#         'CoT-EM by Category': cot_em_by_category,
#         'CoT-Partial by Category': cot_partial_by_category,
#         'Total Questions Evaluated for Step Verification': total_questions
#     }
#     return results


# def compute_cot_hop_em(df: pd.DataFrame) -> float: # Strict CoT EM
#     exact_matches = 0
#     total = 0
#     for _, row in df.iterrows():
#         gold_hops = row.get('gold_hops', [])
#         pred_hops = row.get('model_reasoning_steps', [])
        
#         if not gold_hops or not pred_hops or not pred_hops[0]: # ensure pred_hops is not empty or has empty first step
#             continue
            
#         total += 1
#         # Compare only up to the length of the shorter list of hops,
#         # but for strict EM, lengths must match.
#         if len(pred_hops) == len(gold_hops) and \
#            all(g.strip().lower() == p.strip().lower() for g, p in zip(gold_hops, pred_hops)):
#             exact_matches += 1
            
#     cot_em = exact_matches / total if total > 0 else 0.0
#     print(f"Strict CoT-EM (exact hop sequence): {cot_em:.4f} ({exact_matches}/{total})")
#     return cot_em

# def compute_cot_hop_em_by_category(df: pd.DataFrame) -> dict:
#     results = {}
#     # Ensure categories are sorted for consistent output
#     sorted_categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
#     for cat in sorted_categories:
#         subdf = df[df['hop_category'] == cat]
#         if subdf.empty:
#             results[str(cat)] = 0.0
#             continue
#         em = compute_cot_hop_em(subdf) # This will print for each category, which might be verbose.
#                                       # For aggregated report, better to compute silently.
#         # Silent computation for aggregation:
#         exact_matches = 0
#         total = 0
#         for _, row in subdf.iterrows():
#             gold_hops = row.get('gold_hops', [])
#             pred_hops = row.get('model_reasoning_steps', [])
#             if not gold_hops or not pred_hops or not pred_hops[0]: continue
#             total += 1
#             if len(pred_hops) == len(gold_hops) and \
#                all(g.strip().lower() == p.strip().lower() for g, p in zip(gold_hops, pred_hops)):
#                 exact_matches += 1
#         results[str(cat)] = exact_matches / total if total > 0 else 0.0
#     return results

# # --- VISUALIZATION ---

# def visualize_hop_wise_results(hop_results, step_results, strict_cot_em_overall, cot_em_per_cat_strict, save_path=None):
#     plt.style.use('seaborn-v0_8-whitegrid') # Using a seaborn style
#     plt.figure(figsize=(15, 9)) # Adjusted figure size
    
#     # Ensure categories are sorted as in hop_results (which should be from evaluate_hop_wise_accuracy)
#     categories = [cat for cat in hop_results.keys() if cat != 'overall']
    
#     counts = [hop_results[cat]['count'] for cat in categories]
#     em_scores = [hop_results[cat]['EM'] for cat in categories]
#     f1_scores = [hop_results[cat]['F1'] for cat in categories]
#     bleu_scores = [hop_results[cat]['BLEU'] for cat in categories] # Get BLEU scores
    
#     soft_cot_em_by_cat = step_results.get('CoT-EM by Category', {})
#     soft_cot_em_scores = [soft_cot_em_by_cat.get(cat, 0) for cat in categories]
    
#     # cot_em_per_cat_strict is already the dict from compute_cot_hop_em_by_category
#     strict_cot_em_scores = [cot_em_per_cat_strict.get(cat, 0) for cat in categories]
    
#     x = np.arange(len(categories))
#     width = 0.15 # Width for each bar (5 bars per group)
    
#     plt.bar(x - width*2, em_scores, width, label='Exact Match (Answer)', color='#3498db', alpha=0.85, edgecolor='black', linewidth=0.5)
#     plt.bar(x - width, f1_scores, width, label='F1 Score (Answer)', color='#2ecc71', alpha=0.85, edgecolor='black', linewidth=0.5)
#     plt.bar(x, bleu_scores, width, label='BLEU Score (Answer)', color='#f39c12', alpha=0.85, edgecolor='black', linewidth=0.5)
#     plt.bar(x + width, soft_cot_em_scores, width, label='Soft CoT-EM (Path)', color='#e74c3c', alpha=0.85, edgecolor='black', linewidth=0.5)
#     plt.bar(x + width*2, strict_cot_em_scores, width, label='Strict CoT-EM (Path)', color='#8e44ad', alpha=0.85, edgecolor='black', linewidth=0.5)
    
#     plt.xlabel('Number of Reasoning Hops (Heuristic)', fontsize=13, fontweight='bold')
#     plt.ylabel('Score', fontsize=13, fontweight='bold')
#     plt.title('Qwen3-0.6B Performance on MenatQA by Reasoning Complexity', fontsize=16, fontweight='bold', pad=20)
#     plt.xticks(x, categories, fontsize=11)
#     plt.yticks(fontsize=10)
#     plt.ylim(0, 1.05)
    
#     for i, count in enumerate(counts):
#         if count > 0: # Only add text if there are samples
#             plt.text(i, 0.015, f'n={count}', ha='center', va='bottom', fontsize=9,
#                      bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.2'))
                     
#     overall = hop_results['overall']
#     overall_text = (f"Overall Metrics (n={overall['count']}):\n"
#                     f"Answer EM: {overall['EM']:.3f}  |  Answer F1: {overall['F1']:.3f}  |  Answer BLEU: {overall['BLEU']:.3f}\n"
#                     f"Path Soft CoT-EM: {step_results['CoT-EM']:.3f}  |  Path Strict CoT-EM: {strict_cot_em_overall:.3f}")
                    
#     plt.figtext(0.5, 0.005, overall_text, ha='center', fontsize=11, fontweight='bold',
#                 bbox=dict(facecolor='whitesmoke', edgecolor='gray', boxstyle='round,pad=0.5'))
                
#     plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.12), ncol=3, fontsize=10, frameon=True, edgecolor='black')
#     plt.grid(axis='y', linestyle='--', alpha=0.7)
#     plt.tight_layout(rect=[0, 0.06, 1, 0.95]) # Adjust layout to make space for figtext and title
    
#     if save_path:
#         plt.savefig(save_path, dpi=300, bbox_inches='tight')
#         print(f"Visualization saved to {save_path}")
#     plt.show()

# # --- MAIN ---
# if __name__ == '__main__':
#     # Ensure NLTK data is available if more advanced tokenization were used in compute_bleu
#     # For current simple .split(), it's not strictly needed for BLEU but good for other NLTK uses.
#     # import nltk
#     # try:
#     #     nltk.data.find('tokenizers/punkt')
#     # except nltk.downloader.DownloadError:
#     #     print("NLTK 'punkt' resource not found. Downloading...")
#     #     nltk.download('punkt', quiet=True)


#     # --- Configuration ---
#     DATASET_FILE = '/content/MenatQA.json'
#     MODEL_NAME = "Qwen/Qwen3-0.6B" # "Qwen/Qwen1.5-0.5B-Chat" # or other small models
#     OUTPUT_IMAGE_PATH = '/content/menatqa_qwen_performance.png'
#     OUTPUT_RESULTS_PATH = '/content/menatqa_qwen_summary_results.json'
#     # For faster testing, you can subset the data:
#     # MAX_SAMPLES = 10 # Set to None to run on all data
#     MAX_SAMPLES = None


#     # --- Execution ---
#     data = load_menatqa_dataset(DATASET_FILE)
    
#     if MAX_SAMPLES is not None:
#         print(f"Subsetting dataset to {MAX_SAMPLES} samples for quick testing.")
#         data = data[:MAX_SAMPLES]

#     df = preprocess_dataset(data)

#     print(f"\nRunning {MODEL_NAME} on {len(df)} questions. This may take time.")
#     df_results = run_qwen_cot_predictions(df, model_name=MODEL_NAME)

#     print("\nEvaluating results...")
#     hop_results = evaluate_hop_wise_accuracy(df_results)
#     step_verification_results = evaluate_step_verification(df_results)
#     strict_cot_em_overall = compute_cot_hop_em(df_results) # This will print the overall strict CoT-EM
#     strict_cot_em_by_category = compute_cot_hop_em_by_category(df_results) # This will compute per category for the dict

#     print("\n--- Hop-wise Answer Accuracy (including BLEU) ---")
#     for category, metrics in hop_results.items():
#         if category == 'overall': continue
#         print(f"Category: {category} (n={metrics['count']}) | EM: {metrics['EM']:.3f}, F1: {metrics['F1']:.3f}, BLEU: {metrics['BLEU']:.3f}")
#     print(f"Overall (n={hop_results['overall']['count']}): EM: {hop_results['overall']['EM']:.3f}, F1: {hop_results['overall']['F1']:.3f}, BLEU: {hop_results['overall']['BLEU']:.3f}")

#     print("\n--- Reasoning Path Verification ---")
#     print(f"Overall Soft CoT-EM (Path): {step_verification_results['CoT-EM']:.3f}")
#     print(f"Overall CoT-Partial (Path): {step_verification_results['CoT-Partial']:.3f}")
#     print(f"Overall Hop-Match-Rate: {step_verification_results['Hop-Match-Rate']:.3f}")
#     print(f"Overall Strict CoT-EM (Path): {strict_cot_em_overall:.3f}")
    
#     print("\nBy Category - Soft CoT-EM (Path):")
#     for cat, score in step_verification_results['CoT-EM by Category'].items():
#         print(f"  {cat}: {score:.3f}")
#     print("By Category - Strict CoT-EM (Path):")
#     for cat, score in strict_cot_em_by_category.items():
#         print(f"  {cat}: {score:.3f}")


#     visualize_hop_wise_results(
#         hop_results,
#         step_verification_results,
#         strict_cot_em_overall,
#         strict_cot_em_by_category,
#         save_path=OUTPUT_IMAGE_PATH
#     )

#     summary_results_output = {
#         'model_name': MODEL_NAME,
#         'dataset_info': {
#             'name': 'MenatQA',
#             'total_questions_processed': len(df_results),
#             'hop_distribution': df_results['hop_category'].value_counts().to_dict()
#         },
#         'answer_accuracy_metrics': hop_results,
#         'reasoning_path_verification_metrics': step_verification_results,
#         'strict_cot_em_overall': float(strict_cot_em_overall),
#         'strict_cot_em_by_category': {k: float(v) for k, v in strict_cot_em_by_category.items()}
#     }
#     with open(OUTPUT_RESULTS_PATH, 'w') as f:
#         json.dump(summary_results_output, f, indent=2)

#     print(f"\nEvaluation complete! Results saved to {OUTPUT_RESULTS_PATH} and {OUTPUT_IMAGE_PATH}")

In [ ]:
# import json
# import re
# import os
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from collections import defaultdict
# from tqdm import tqdm
# import torch
# from pathlib import Path

# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# # Download required NLTK data
# import nltk
# try:
#     nltk.data.find('tokenizers/punkt')
# except LookupError:
#     print("Downloading required NLTK data...")
#     nltk.download('punkt', quiet=True)

# # --- CONFIGURATION ---
# class EvaluationConfig:
#     def __init__(self):
#         # Create output directory in current working directory
#         self.output_dir = Path.cwd() / "menatqa_evaluation"
#         self.output_dir.mkdir(exist_ok=True)
        
#         self.dataset_file = self.output_dir / 'MenatQA.json'
#         self.model_name = "Qwen/Qwen3-0.6B"  # Updated to available model
#         self.max_samples = None  # Set to small number for testing
#         self.max_new_tokens = 256
#         self.temperature = 0.0
#         self.batch_size = 1
#         self.use_fp16 = True
        
#         # Output files
#         self.output_image_path = self.output_dir / 'menatqa_performance.png'
#         self.output_results_path = self.output_dir / 'menatqa_results.json'
        
#     def to_dict(self):
#         # Convert Path objects to strings for JSON serialization
#         config_dict = {}
#         for key, value in self.__dict__.items():
#             if isinstance(value, Path):
#                 config_dict[key] = str(value)
#             else:
#                 config_dict[key] = value
#         return config_dict

# # --- DATA ---
# def load_menatqa_dataset(file_path):
#     """Load MenatQA dataset with improved error handling"""
#     file_path = Path(file_path)
    
#     # Create parent directory if it doesn't exist
#     file_path.parent.mkdir(parents=True, exist_ok=True)
    
#     if not file_path.exists():
#         import urllib.request
#         print(f"Downloading MenatQA to {file_path}...")
#         try:
#             urllib.request.urlretrieve(
#                 "https://raw.githubusercontent.com/weiyifan1023/MenatQA/main/datasets/MenatQA.json",
#                 str(file_path)
#             )
#             print("Download completed successfully!")
#         except Exception as e:
#             print(f"Error downloading dataset: {e}")
#             # Fallback: create a small sample dataset for testing
#             sample_data = [
#                 {
#                     "ID": "test_1",
#                     "question": "What is the capital of France?",
#                     "answer": "Paris",
#                     "type": "factual",
#                     "time_scope": ""
#                 },
#                 {
#                     "ID": "test_2", 
#                     "question": "Who was the first president of the United States and when did he serve?",
#                     "answer": "George Washington served from 1789 to 1797",
#                     "type": "historical",
#                     "time_scope": "1789-1797"
#                 }
#             ]
#             with open(file_path, 'w') as f:
#                 json.dump(sample_data, f, indent=2)
#             print(f"Created sample dataset at {file_path}")
    
#     with open(file_path, 'r') as f:
#         data = json.load(f)
#     print(f"Loaded MenatQA dataset with {len(data)} examples")
#     return data

# def extract_reasoning_hops_advanced(example):
#     """Enhanced hop detection with better linguistic analysis"""
#     question = example.get('question', '')
#     answer = example.get('answer', '')
#     q_type = example.get('type', '')
#     time_scope = example.get('time_scope', '')
    
#     # More sophisticated analysis
#     sentences = [s.strip() for s in question.split('.') if s.strip()]
    
#     # Enhanced logical connector detection
#     logical_connectors = ['and', 'or', 'but', 'because', 'since', 'although', 
#                          'however', 'therefore', 'moreover', 'furthermore', 'while']
#     temporal_words = ['before', 'after', 'during', 'when', 'then', 'until', 'since']
    
#     connector_count = sum(1 for word in logical_connectors if word in question.lower())
#     temporal_count = sum(1 for word in temporal_words if word in question.lower())
    
#     # Question complexity indicators
#     question_words = ['what', 'where', 'when', 'who', 'why', 'how', 'which']
#     q_word_count = sum(1 for word in question_words if word in question.lower())
    
#     capitalized_words = len([w for w in question.split() if w and w[0].isupper()])
    
#     # Enhanced complexity scoring
#     complexity_score = 1
#     complexity_score += min(2, len(sentences) - 1)  # Multiple sentences
#     complexity_score += min(1, connector_count)     # Logical connections
#     complexity_score += min(1, temporal_count)      # Temporal reasoning
#     complexity_score += min(1, q_word_count - 1)    # Multiple question aspects
#     complexity_score += min(1, capitalized_words // 4)  # Named entities
    
#     if time_scope:
#         complexity_score += 1
    
#     complexity_score = min(4, max(1, complexity_score))
    
#     # Generate reasoning hops
#     hops = []
#     hops.append(f"Understand the question: {question}")
    
#     if complexity_score >= 2:
#         if time_scope:
#             hops.append(f"Identify temporal context: {time_scope}")
#         elif connector_count > 0:
#             hops.append("Analyze logical relationships in the question")
#         else:
#             hops.append(f"Recognize question type: {q_type}")
    
#     if complexity_score >= 3:
#         hops.append("Retrieve and analyze relevant facts")
        
#     if complexity_score >= 4:
#         hops.append("Synthesize information and resolve dependencies")
    
#     hops.append(f"Formulate final answer: {answer}")
    
#     return hops, complexity_score

# def preprocess_dataset(data):
#     """Preprocess dataset with enhanced hop extraction"""
#     processed_data = []
#     hop_counts = {1: 0, 2: 0, 3: 0, 4: 0}
    
#     for item in data:
#         gold_hops, complexity_score = extract_reasoning_hops_advanced(item)
#         hop_counts[min(4, complexity_score)] += 1
        
#         entry = {
#             'ID': item.get('ID', ''),
#             'question': item.get('question', ''),
#             'answer': item.get('answer', ''),
#             'type': item.get('type', ''),
#             'time_scope': item.get('time_scope', ''),
#             'gold_hops': gold_hops,
#             'hop_count': complexity_score,
#             'model_prediction': '',
#             'model_reasoning_steps': []
#         }
#         processed_data.append(entry)
    
#     df = pd.DataFrame(processed_data)
#     df['hop_category'] = pd.cut(
#         df['hop_count'],
#         bins=[-1, 1, 2, 3, float('inf')],
#         labels=['1-hop', '2-hop', '3-hop', '4+-hop']
#     )
    
#     print(f"Dataset preprocessed: {len(df)} questions")
#     for i in range(1, 5):
#         print(f"{i}-hop questions: {hop_counts[i]} ({hop_counts[i]/len(df):.1%})")
    
#     return df

# # --- LLM ---
# def run_qwen_cot_predictions(df, config):
#     """Enhanced model inference with better memory management"""
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     print(f"Loading model {config.model_name} on {device}...")
    
#     # Clear GPU memory
#     if device == "cuda":
#         torch.cuda.empty_cache()
    
#     try:
#         tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        
#         # Memory-efficient model loading
#         model_kwargs = {
#             'torch_dtype': torch.float16 if config.use_fp16 and device == "cuda" else 'auto',
#             'device_map': "auto" if device == "cuda" else None
#         }
        
#         model = AutoModelForCausalLM.from_pretrained(config.model_name, **model_kwargs)
#         if not model_kwargs.get('device_map'):
#             model = model.to(device)
            
#         # Create pipeline
#         pipe = pipeline(
#             "text-generation", 
#             model=model, 
#             tokenizer=tokenizer, 
#             device=0 if device == 'cuda' else -1,
#             max_new_tokens=config.max_new_tokens,
#             temperature=config.temperature,
#             do_sample=False
#         )
        
#     except Exception as e:
#         print(f"Error loading model: {e}")
#         print("Using dummy predictions for testing...")
#         return create_dummy_predictions(df)

#     def format_cot_prompt(question):
#         return (f"Answer the following question step by step. "
#                 f"Write each reasoning step on a new line, and finish with your answer.\n"
#                 f"Question: {question}\nLet's think step by step:")

#     all_preds, all_steps = [], []
    
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating predictions"):
#         prompt = format_cot_prompt(row["question"])
        
#         try:
#             generation = pipe(prompt)[0]['generated_text']
#             output = generation[len(prompt):] if generation.startswith(prompt) else generation
#             steps = [x.strip() for x in output.split('\n') if x.strip()]
            
#             if not steps:
#                 steps = ["Unable to generate reasoning steps"]
                
#             model_answer = steps[-1] if steps else "No answer generated"
            
#         except Exception as e:
#             print(f"Error on question {idx}: {e}")
#             steps = ["Error in generation"]
#             model_answer = "Error"
        
#         all_preds.append(model_answer)
#         all_steps.append(steps)
    
#     df = df.copy()
#     df['model_prediction'] = all_preds
#     df['model_reasoning_steps'] = all_steps
    
#     return df

# def create_dummy_predictions(df):
#     """Create dummy predictions for testing when model loading fails"""
#     dummy_preds = []
#     dummy_steps = []
    
#     for _, row in df.iterrows():
#         # Create plausible dummy reasoning
#         steps = [
#             f"I need to understand: {row['question'][:50]}...",
#             "Let me think about the relevant information.",
#             "Based on my analysis, the answer is:"
#         ]
#         pred = f"Dummy answer for: {row['question'][:20]}..."
        
#         dummy_preds.append(pred)
#         dummy_steps.append(steps)
    
#     df = df.copy()
#     df['model_prediction'] = dummy_preds
#     df['model_reasoning_steps'] = dummy_steps
    
#     print("Created dummy predictions for testing")
#     return df

# # --- ENHANCED METRICS ---
# def compute_exact_match(prediction, gold):
#     """Enhanced exact match with better preprocessing"""
#     pred_clean = str(prediction).strip().lower()
#     gold_clean = str(gold).strip().lower()
    
#     # Remove common prefixes/suffixes
#     prefixes = ["the answer is", "answer:", "final answer:"]
#     for prefix in prefixes:
#         if pred_clean.startswith(prefix):
#             pred_clean = pred_clean[len(prefix):].strip()
    
#     return int(pred_clean == gold_clean)

# def compute_f1(prediction, gold):
#     """Enhanced F1 with better tokenization"""
#     pred_tokens = str(prediction).lower().split()
#     gold_tokens = str(gold).lower().split()
    
#     # Remove stopwords for better matching
#     stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were'}
#     pred_tokens = [t for t in pred_tokens if t not in stopwords]
#     gold_tokens = [t for t in gold_tokens if t not in stopwords]
    
#     common = set(pred_tokens) & set(gold_tokens)
    
#     if not pred_tokens or not gold_tokens:
#         return 0.0
    
#     prec = len(common) / len(pred_tokens)
#     rec = len(common) / len(gold_tokens)
    
#     if prec + rec == 0:
#         return 0.0
    
#     return 2 * prec * rec / (prec + rec)

# def compute_bleu(prediction, gold):
#     """Enhanced BLEU computation"""
#     pred_tokens = str(prediction).lower().split()
#     gold_tokens = str(gold).lower().split()

#     if not pred_tokens or not gold_tokens:
#         return 0.0

#     reference = [gold_tokens]
#     candidate = pred_tokens
    
#     weights = (0.25, 0.25, 0.25, 0.25)
#     chencherry = SmoothingFunction()
    
#     try:
#         bleu_score = sentence_bleu(
#             reference, candidate, 
#             weights=weights, 
#             smoothing_function=chencherry.method1
#         )
#     except (ZeroDivisionError, ValueError):
#         bleu_score = 0.0
    
#     return bleu_score

# # [Keep all the existing evaluation functions: evaluate_hop_wise_accuracy, evaluate_step_verification, etc.]
# def evaluate_hop_wise_accuracy(df):
#     categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
#     hop_results = {}
#     overall_em, overall_f1, overall_bleu, overall_count = 0, 0, 0, 0
    
#     for cat in categories:
#         subdf = df[df['hop_category'] == cat]
#         if subdf.empty:
#             hop_results[str(cat)] = {'count': 0, 'EM': 0, 'F1': 0, 'BLEU': 0}
#             continue

#         ems = [compute_exact_match(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
#         f1s = [compute_f1(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
#         bleus = [compute_bleu(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
        
#         count = len(subdf)
#         mean_em = np.mean(ems) if ems else 0
#         mean_f1 = np.mean(f1s) if f1s else 0
#         mean_bleu = np.mean(bleus) if bleus else 0
        
#         hop_results[str(cat)] = {'count': count, 'EM': mean_em, 'F1': mean_f1, 'BLEU': mean_bleu}
        
#         overall_em += sum(ems)
#         overall_f1 += sum(f1s)
#         overall_bleu += sum(bleus)
#         overall_count += count
        
#     hop_results['overall'] = {
#         'count': overall_count,
#         'EM': overall_em / overall_count if overall_count else 0,
#         'F1': overall_f1 / overall_count if overall_count else 0,
#         'BLEU': overall_bleu / overall_count if overall_count else 0,
#     }
#     return hop_results

# def evaluate_step_verification(df: pd.DataFrame) -> dict:
#     all_step_matches = []
#     complete_path_matches = 0
#     partial_path_matches = 0
#     total_questions = 0
#     sorted_categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
#     hop_category_results = defaultdict(lambda: {'matches': 0, 'partial_matches': 0, 'total': 0})

#     for _, row in df.iterrows():
#         gold_hops = row['gold_hops']
#         model_steps = row['model_reasoning_steps']
        
#         if not gold_hops or not model_steps or not model_steps[0]:
#             continue
            
#         total_questions += 1
#         hop_cat = row['hop_category']
#         if hop_cat is None: continue

#         hop_category_results[hop_cat]['total'] += 1
        
#         step_matches = []
#         first_match = 0
#         last_match = 0
#         middle_matches = []

#         # First step (question understanding)
#         gold_q_terms = set(re.findall(r'\w+', gold_hops[0].lower()))
#         model_q_terms = set(re.findall(r'\w+', model_steps[0].lower()))
#         if gold_q_terms:
#             common_terms = gold_q_terms & model_q_terms
#             first_match = len(common_terms) / len(gold_q_terms)
#         step_matches.append(first_match)

#         # Last step (answer formulation)
#         if len(gold_hops) > 1 and len(model_steps) > 1:
#             gold_ans_text = row['answer'].lower()
#             model_last_hop_text = model_steps[-1].lower()
            
#             # Check if the gold answer appears in the model's final step
#             last_match = 1.0 if gold_ans_text in model_last_hop_text else 0.0
#             step_matches.append(last_match)

#         # Intermediate steps (middle reasoning)
#         if len(gold_hops) > 2 and len(model_steps) > 2:
#             # Compare intermediate steps (excluding first and last)
#             num_intermediate_to_compare = min(len(gold_hops) - 2, len(model_steps) - 2)
            
#             for i in range(num_intermediate_to_compare):
#                 gold_mid_idx = i + 1  # Skip first step
#                 model_mid_idx = i + 1  # Skip first step
                
#                 gold_mid = gold_hops[gold_mid_idx].lower()
#                 model_mid = model_steps[model_mid_idx].lower()
                
#                 # Extract meaningful words from both steps
#                 gold_words = set(re.findall(r'\w+', gold_mid))
#                 model_words = set(re.findall(r'\w+', model_mid))
                
#                 # Calculate word overlap similarity
#                 if gold_words and model_words:
#                     common_words = gold_words & model_words
#                     # Use Jaccard similarity
#                     similarity = len(common_words) / max(len(gold_words), len(model_words))
#                     middle_matches.append(similarity)
#                     step_matches.append(similarity)
#                 elif gold_words or model_words:
#                     # One is empty, the other is not
#                     middle_matches.append(0.0)
#                     step_matches.append(0.0)
#                 # If both are empty, skip this comparison

#         # Add all step matches to overall tracking
#         all_step_matches.extend(step_matches)
        
#         # Determine match quality
#         if step_matches:
#             # Complete match: all compared steps must meet high threshold
#             is_complete_match = all(match >= 0.6 for match in step_matches)

#             # Partial match: critical steps (first and last) are good, middle is reasonable
#             is_partial_match = False
#             if not is_complete_match:
#                 # Calculate average middle step performance
#                 avg_middle_match = sum(middle_matches) / len(middle_matches) if middle_matches else 1.0
                
#                 # Partial match criteria: good first/last steps, reasonable middle steps
#                 if first_match >= 0.7 and last_match >= 0.7 and avg_middle_match >= 0.4:
#                     is_partial_match = True
            
#             # Update counters
#             if is_complete_match:
#                 complete_path_matches += 1
#                 hop_category_results[hop_cat]['matches'] += 1
#             elif is_partial_match:
#                 partial_path_matches += 1
#                 hop_category_results[hop_cat]['partial_matches'] += 1

#     # Calculate overall metrics
#     cot_em_overall = complete_path_matches / total_questions if total_questions > 0 else 0
#     cot_partial_overall = (complete_path_matches + partial_path_matches) / total_questions if total_questions > 0 else 0
    
#     # Calculate by-category metrics
#     cot_em_by_category = {}
#     cot_partial_by_category = {}

#     for hop_cat_key in sorted_categories:
#         data = hop_category_results[hop_cat_key]
#         cat_total = data['total']
#         cot_em = data['matches'] / cat_total if cat_total > 0 else 0
#         cot_partial = (data['matches'] + data['partial_matches']) / cat_total if cat_total > 0 else 0
#         cot_em_by_category[str(hop_cat_key)] = cot_em
#         cot_partial_by_category[str(hop_cat_key)] = cot_partial
        
#     # Return comprehensive results
#     results = {
#         'CoT-EM': cot_em_overall,
#         'CoT-Partial': cot_partial_overall,
#         'Hop-Match-Rate': np.mean(all_step_matches) if all_step_matches else 0,
#         'CoT-EM by Category': cot_em_by_category,
#         'CoT-Partial by Category': cot_partial_by_category,
#         'Total Questions Evaluated for Step Verification': total_questions,
#         'Complete Path Matches': complete_path_matches,
#         'Partial Path Matches': partial_path_matches
#     }
#     return results


# def compute_cot_hop_em(df: pd.DataFrame) -> float:
#     """Strict CoT EM - exact hop sequence matching"""
#     exact_matches = 0
#     total = 0
#     for _, row in df.iterrows():
#         gold_hops = row.get('gold_hops', [])
#         pred_hops = row.get('model_reasoning_steps', [])
        
#         if not gold_hops or not pred_hops or not pred_hops[0]:
#             continue
            
#         total += 1
#         if len(pred_hops) == len(gold_hops) and \
#            all(g.strip().lower() == p.strip().lower() for g, p in zip(gold_hops, pred_hops)):
#             exact_matches += 1
            
#     cot_em = exact_matches / total if total > 0 else 0.0
#     return cot_em

# def compute_cot_hop_em_by_category(df: pd.DataFrame) -> dict:
#     """Compute strict CoT-EM by hop category"""
#     results = {}
#     sorted_categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
    
#     for cat in sorted_categories:
#         subdf = df[df['hop_category'] == cat]
#         if subdf.empty:
#             results[str(cat)] = 0.0
#             continue
            
#         exact_matches = 0
#         total = 0
#         for _, row in subdf.iterrows():
#             gold_hops = row.get('gold_hops', [])
#             pred_hops = row.get('model_reasoning_steps', [])
#             if not gold_hops or not pred_hops or not pred_hops[0]: 
#                 continue
#             total += 1
#             if len(pred_hops) == len(gold_hops) and \
#                all(g.strip().lower() == p.strip().lower() for g, p in zip(gold_hops, pred_hops)):
#                 exact_matches += 1
                
#         results[str(cat)] = exact_matches / total if total > 0 else 0.0
#     return results

# # --- ENHANCED VISUALIZATION ---
# def visualize_hop_wise_results(hop_results, step_results, strict_cot_em_overall, cot_em_per_cat_strict, config):
#     """Enhanced visualization with better styling"""
#     plt.style.use('default')  # Use default style for compatibility
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
#     # Main performance chart
#     categories = [cat for cat in hop_results.keys() if cat != 'overall']
#     counts = [hop_results[cat]['count'] for cat in categories]
#     em_scores = [hop_results[cat]['EM'] for cat in categories]
#     f1_scores = [hop_results[cat]['F1'] for cat in categories]
#     bleu_scores = [hop_results[cat]['BLEU'] for cat in categories]
    
#     soft_cot_em_by_cat = step_results.get('CoT-EM by Category', {})
#     soft_cot_em_scores = [soft_cot_em_by_cat.get(cat, 0) for cat in categories]
#     strict_cot_em_scores = [cot_em_per_cat_strict.get(cat, 0) for cat in categories]
    
#     x = np.arange(len(categories))
#     width = 0.15
    
#     ax1.bar(x - width*2, em_scores, width, label='Exact Match', color='#3498db', alpha=0.8)
#     ax1.bar(x - width, f1_scores, width, label='F1 Score', color='#2ecc71', alpha=0.8)
#     ax1.bar(x, bleu_scores, width, label='BLEU Score', color='#f39c12', alpha=0.8)
#     ax1.bar(x + width, soft_cot_em_scores, width, label='Soft CoT-EM', color='#e74c3c', alpha=0.8)
#     ax1.bar(x + width*2, strict_cot_em_scores, width, label='Strict CoT-EM', color='#8e44ad', alpha=0.8)
    
#     ax1.set_xlabel('Reasoning Complexity', fontweight='bold')
#     ax1.set_ylabel('Score', fontweight='bold')
#     ax1.set_title(f'{config.model_name} Performance by Complexity', fontweight='bold', pad=20)
#     ax1.set_xticks(x)
#     ax1.set_xticklabels(categories)
#     ax1.set_ylim(0, 1.05)
#     ax1.legend(loc='upper right')
#     ax1.grid(axis='y', alpha=0.3)
    
#     # Add sample counts
#     for i, count in enumerate(counts):
#         if count > 0:
#             ax1.text(i, 0.02, f'n={count}', ha='center', va='bottom', fontsize=9,
#                     bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.2'))
    
#     # Distribution pie chart
#     if counts and sum(counts) > 0:
#         ax2.pie(counts, labels=categories, autopct='%1.1f%%', startangle=90)
#         ax2.set_title('Question Distribution by Complexity', fontweight='bold')
#     else:
#         ax2.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=ax2.transAxes)
#         ax2.set_title('No Distribution Data', fontweight='bold')
    
#     # Overall metrics text
#     overall = hop_results['overall']
#     overall_text = (f"Overall Results (n={overall['count']}):\n"
#                    f"EM: {overall['EM']:.3f} | F1: {overall['F1']:.3f} | BLEU: {overall['BLEU']:.3f}\n"
#                    f"Soft CoT-EM: {step_results['CoT-EM']:.3f} | Strict CoT-EM: {strict_cot_em_overall:.3f}")
    
#     fig.text(0.5, 0.02, overall_text, ha='center', fontsize=11, fontweight='bold',
#              bbox=dict(facecolor='lightgray', alpha=0.8, boxstyle='round,pad=0.5'))
    
#     plt.tight_layout(rect=[0, 0.1, 1, 0.96])
    
#     # Save plot
#     plt.savefig(config.output_image_path, dpi=300, bbox_inches='tight')
#     print(f"Visualization saved to {config.output_image_path}")
#     plt.show()

# # --- MAIN EXECUTION ---
# def main():
#     """Main execution function with comprehensive error handling"""
#     print("=== MenatQA Evaluation Pipeline ===\n")
    
#     # Initialize configuration
#     config = EvaluationConfig()
#     print(f"Output directory: {config.output_dir}")
#     print(f"Model: {config.model_name}")
#     print(f"Max samples: {config.max_samples or 'All'}\n")
    
#     try:
#         # Load and preprocess data
#         print("📊 Loading dataset...")
#         data = load_menatqa_dataset(config.dataset_file)
        
#         if config.max_samples is not None:
#             print(f"Limiting to {config.max_samples} samples for testing")
#             data = data[:config.max_samples]
        
#         print("🔄 Preprocessing dataset...")
#         df = preprocess_dataset(data)
        
#         # Run model inference
#         print(f"\n🤖 Running {config.model_name} inference...")
#         df_results = run_qwen_cot_predictions(df, config)
        
#         # Evaluate results
#         print("\n📈 Evaluating results...")
#         hop_results = evaluate_hop_wise_accuracy(df_results)
#         step_verification_results = evaluate_step_verification(df_results)
#         strict_cot_em_overall = compute_cot_hop_em(df_results)
#         strict_cot_em_by_category = compute_cot_hop_em_by_category(df_results)
        
#         # Display results
#         print("\n" + "="*60)
#         print("📊 RESULTS SUMMARY")
#         print("="*60)
        
#         print("\n--- Answer Accuracy by Complexity ---")
#         for category, metrics in hop_results.items():
#             if category == 'overall': continue
#             print(f"{category:>10} (n={metrics['count']:>3}): EM={metrics['EM']:.3f}, F1={metrics['F1']:.3f}, BLEU={metrics['BLEU']:.3f}")
        
#         overall = hop_results['overall']
#         print(f"{'Overall':>10} (n={overall['count']:>3}): EM={overall['EM']:.3f}, F1={overall['F1']:.3f}, BLEU={overall['BLEU']:.3f}")
        
#         print("\n--- Reasoning Path Verification ---")
#         print(f"Soft CoT-EM (Path):   {step_verification_results['CoT-EM']:.3f}")
#         print(f"CoT-Partial (Path):   {step_verification_results['CoT-Partial']:.3f}")
#         print(f"Hop-Match-Rate:       {step_verification_results['Hop-Match-Rate']:.3f}")
#         print(f"Strict CoT-EM (Path): {strict_cot_em_overall:.3f}")
        
#         print("\nBy Category - Soft CoT-EM:")
#         for cat, score in step_verification_results['CoT-EM by Category'].items():
#             print(f"  {cat}: {score:.3f}")
        
#         # Generate visualization
#         print("\n📊 Generating visualization...")
#         visualize_hop_wise_results(
#             hop_results, step_verification_results, 
#             strict_cot_em_overall, strict_cot_em_by_category, config
#         )
        
#         # Save comprehensive results
#         summary_results = {
#             'config': config.to_dict(),
#             'dataset_info': {
#                 'name': 'MenatQA',
#                 'total_questions_processed': len(df_results),
#                 'hop_distribution': df_results['hop_category'].value_counts().to_dict()
#             },
#             'answer_accuracy_metrics': hop_results,
#             'reasoning_path_verification_metrics': step_verification_results,
#             'strict_cot_em_overall': float(strict_cot_em_overall),
#             'strict_cot_em_by_category': {k: float(v) for k, v in strict_cot_em_by_category.items()}
#         }
        
#         with open(config.output_results_path, 'w') as f:
#             json.dump(summary_results, f, indent=2, default=str)
        
#         print(f"\n✅ Evaluation complete!")
#         print(f"📁 Results saved to: {config.output_results_path}")
#         print(f"📊 Visualization saved to: {config.output_image_path}")
        
#         return df_results, summary_results
        
#     except Exception as e:
#         print(f"\n❌ Error during evaluation: {e}")
#         print("This might be due to model loading issues. Check your GPU/CPU setup.")
#         return None, None

# if __name__ == '__main__':
#     # Set test configuration for initial run
#     config = EvaluationConfig()
#     config.max_samples = 10  # Small test run
#     config.model_name = "Qwen/Qwen3-0.6B"  # Try smaller model first
    
#     print("🧪 Running test evaluation with limited samples...")
#     main()

**NEW CODE**

In [ ]:
# import json
# import re
# import os
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from collections import defaultdict
# from tqdm import tqdm
# import torch
# from pathlib import Path

# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# # Download required NLTK data
# import nltk
# try:
#     nltk.data.find('tokenizers/punkt')
# except LookupError:
#     print("Downloading required NLTK data...")
#     nltk.download('punkt', quiet=True)

# # --- DATA ---
# def load_menatqa_dataset(file_path='./MenatQA.json'):
#     """Load MenatQA dataset"""
#     file_path = Path(file_path)
    
#     if not file_path.exists():
#         import urllib.request
#         print(f"Downloading MenatQA to {file_path}...")
#         urllib.request.urlretrieve(
#             "https://raw.githubusercontent.com/weiyifan1023/MenatQA/main/datasets/MenatQA.json",
#             str(file_path)
#         )
    
#     with open(file_path, 'r') as f:
#         data = json.load(f)
#     print(f"Loaded MenatQA dataset with {len(data)} examples")
#     return data

# def extract_reasoning_hops_advanced(example):
#     """Enhanced hop detection with better linguistic analysis"""
#     question = example.get('question', '')
#     answer = example.get('answer', '')
#     q_type = example.get('type', '')
#     time_scope = example.get('time_scope', '')
    
#     # More sophisticated analysis
#     sentences = [s.strip() for s in question.split('.') if s.strip()]
    
#     # Enhanced logical connector detection
#     logical_connectors = ['and', 'or', 'but', 'because', 'since', 'although', 
#                          'however', 'therefore', 'moreover', 'furthermore', 'while']
#     temporal_words = ['before', 'after', 'during', 'when', 'then', 'until', 'since']
    
#     connector_count = sum(1 for word in logical_connectors if word in question.lower())
#     temporal_count = sum(1 for word in temporal_words if word in question.lower())
    
#     # Question complexity indicators
#     question_words = ['what', 'where', 'when', 'who', 'why', 'how', 'which']
#     q_word_count = sum(1 for word in question_words if word in question.lower())
    
#     capitalized_words = len([w for w in question.split() if w and w[0].isupper()])
    
#     # Enhanced complexity scoring
#     complexity_score = 1
#     complexity_score += min(2, len(sentences) - 1)  # Multiple sentences
#     complexity_score += min(1, connector_count)     # Logical connections
#     complexity_score += min(1, temporal_count)      # Temporal reasoning
#     complexity_score += min(1, q_word_count - 1)    # Multiple question aspects
#     complexity_score += min(1, capitalized_words // 4)  # Named entities
    
#     if time_scope:
#         complexity_score += 1
    
#     complexity_score = min(4, max(1, complexity_score))
    
#     # Generate reasoning hops
#     hops = []
#     hops.append(f"Understand the question: {question}")
    
#     if complexity_score >= 2:
#         if time_scope:
#             hops.append(f"Identify temporal context: {time_scope}")
#         elif connector_count > 0:
#             hops.append("Analyze logical relationships in the question")
#         else:
#             hops.append(f"Recognize question type: {q_type}")
    
#     if complexity_score >= 3:
#         hops.append("Retrieve and analyze relevant facts")
        
#     if complexity_score >= 4:
#         hops.append("Synthesize information and resolve dependencies")
    
#     hops.append(f"Formulate final answer: {answer}")
    
#     return hops, complexity_score

# def preprocess_dataset(data):
#     """Preprocess dataset with enhanced hop extraction"""
#     processed_data = []
#     hop_counts = {1: 0, 2: 0, 3: 0, 4: 0}
    
#     for item in data:
#         gold_hops, complexity_score = extract_reasoning_hops_advanced(item)
#         hop_counts[min(4, complexity_score)] += 1
        
#         entry = {
#             'ID': item.get('ID', ''),
#             'question': item.get('question', ''),
#             'answer': item.get('answer', ''),
#             'type': item.get('type', ''),
#             'time_scope': item.get('time_scope', ''),
#             'gold_hops': gold_hops,
#             'hop_count': complexity_score,
#             'model_prediction': '',
#             'model_reasoning_steps': []
#         }
#         processed_data.append(entry)
    
#     df = pd.DataFrame(processed_data)
#     df['hop_category'] = pd.cut(
#         df['hop_count'],
#         bins=[-1, 1, 2, 3, float('inf')],
#         labels=['1-hop', '2-hop', '3-hop', '4+-hop']
#     )
    
#     print(f"Dataset preprocessed: {len(df)} questions")
#     for i in range(1, 5):
#         print(f"{i}-hop questions: {hop_counts[i]} ({hop_counts[i]/len(df):.1%})")
    
#     return df

# # --- MODEL ---
# def run_qwen_predictions(df, model_name="Qwen/Qwen3-0.6B"):
#     """Run Qwen model inference"""
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     print(f"Loading model {model_name} on {device}...")
    
#     # Clear GPU memory
#     if device == "cuda":
#         torch.cuda.empty_cache()
    
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
    
#     # Create pipeline
#     pipe = pipeline(
#         "text-generation", 
#         model=model, 
#         tokenizer=tokenizer, 
#         device=0 if device == 'cuda' else -1,
#         max_new_tokens=256,
#         do_sample=False
#     )

#     def format_cot_prompt(question):
#         return (f"Answer the following question step by step. "
#                 f"Write each reasoning step on a new line, and finish with your answer.\n"
#                 f"Question: {question}\nLet's think step by step:")

#     all_preds, all_steps = [], []
    
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating predictions"):
#         prompt = format_cot_prompt(row["question"])
#         generation = pipe(prompt)[0]['generated_text']
#         output = generation[len(prompt):] if generation.startswith(prompt) else generation
#         steps = [x.strip() for x in output.split('\n') if x.strip()]
        
#         if not steps:
#             steps = [""]
            
#         model_answer = steps[-1]
#         all_preds.append(model_answer)
#         all_steps.append(steps)
    
#     df = df.copy()
#     df['model_prediction'] = all_preds
#     df['model_reasoning_steps'] = all_steps
#     return df

# # --- ENHANCED METRICS ---
# def compute_exact_match(prediction, gold):
#     """Enhanced exact match with better preprocessing"""
#     pred_clean = str(prediction).strip().lower()
#     gold_clean = str(gold).strip().lower()
    
#     # Remove common prefixes/suffixes
#     prefixes = ["the answer is", "answer:", "final answer:"]
#     for prefix in prefixes:
#         if pred_clean.startswith(prefix):
#             pred_clean = pred_clean[len(prefix):].strip()
    
#     return int(pred_clean == gold_clean)

# def compute_f1(prediction, gold):
#     """Enhanced F1 with better tokenization"""
#     pred_tokens = str(prediction).lower().split()
#     gold_tokens = str(gold).lower().split()
    
#     # Remove stopwords for better matching
#     stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were'}
#     pred_tokens = [t for t in pred_tokens if t not in stopwords]
#     gold_tokens = [t for t in gold_tokens if t not in stopwords]
    
#     common = set(pred_tokens) & set(gold_tokens)
    
#     if not pred_tokens or not gold_tokens:
#         return 0.0
    
#     prec = len(common) / len(pred_tokens)
#     rec = len(common) / len(gold_tokens)
    
#     if prec + rec == 0:
#         return 0.0
    
#     return 2 * prec * rec / (prec + rec)

# def compute_bleu(prediction, gold):
#     """Enhanced BLEU computation"""
#     pred_tokens = str(prediction).lower().split()
#     gold_tokens = str(gold).lower().split()

#     if not pred_tokens or not gold_tokens:
#         return 0.0

#     reference = [gold_tokens]
#     candidate = pred_tokens
    
#     weights = (0.25, 0.25, 0.25, 0.25)
#     chencherry = SmoothingFunction()
    
#     try:
#         bleu_score = sentence_bleu(
#             reference, candidate, 
#             weights=weights, 
#             smoothing_function=chencherry.method1
#         )
#     except (ZeroDivisionError, ValueError):
#         bleu_score = 0.0
    
#     return bleu_score

# def evaluate_hop_wise_accuracy(df):
#     categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
#     hop_results = {}
#     overall_em, overall_f1, overall_bleu, overall_count = 0, 0, 0, 0
    
#     for cat in categories:
#         subdf = df[df['hop_category'] == cat]
#         if subdf.empty:
#             hop_results[str(cat)] = {'count': 0, 'EM': 0, 'F1': 0, 'BLEU': 0}
#             continue

#         ems = [compute_exact_match(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
#         f1s = [compute_f1(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
#         bleus = [compute_bleu(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
        
#         count = len(subdf)
#         mean_em = np.mean(ems) if ems else 0
#         mean_f1 = np.mean(f1s) if f1s else 0
#         mean_bleu = np.mean(bleus) if bleus else 0
        
#         hop_results[str(cat)] = {'count': count, 'EM': mean_em, 'F1': mean_f1, 'BLEU': mean_bleu}
        
#         overall_em += sum(ems)
#         overall_f1 += sum(f1s)
#         overall_bleu += sum(bleus)
#         overall_count += count
        
#     hop_results['overall'] = {
#         'count': overall_count,
#         'EM': overall_em / overall_count if overall_count else 0,
#         'F1': overall_f1 / overall_count if overall_count else 0,
#         'BLEU': overall_bleu / overall_count if overall_count else 0,
#     }
#     return hop_results

# def evaluate_step_verification(df):
#     all_step_matches = []
#     complete_path_matches = 0
#     partial_path_matches = 0
#     total_questions = 0
#     sorted_categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
#     hop_category_results = defaultdict(lambda: {'matches': 0, 'partial_matches': 0, 'total': 0})

#     for _, row in df.iterrows():
#         gold_hops = row['gold_hops']
#         model_steps = row['model_reasoning_steps']
        
#         if not gold_hops or not model_steps or not model_steps[0]:
#             continue
            
#         total_questions += 1
#         hop_cat = row['hop_category']
#         if hop_cat is None: continue

#         hop_category_results[hop_cat]['total'] += 1
        
#         step_matches = []
#         first_match = 0
#         last_match = 0
#         middle_matches = []

#         # First step (question understanding)
#         gold_q_terms = set(re.findall(r'\w+', gold_hops[0].lower()))
#         model_q_terms = set(re.findall(r'\w+', model_steps[0].lower()))
#         if gold_q_terms:
#             common_terms = gold_q_terms & model_q_terms
#             first_match = len(common_terms) / len(gold_q_terms)
#         step_matches.append(first_match)

#         # Last step (answer formulation)
#         if len(gold_hops) > 1 and len(model_steps) > 1:
#             gold_ans_text = row['answer'].lower()
#             model_last_hop_text = model_steps[-1].lower()
#             last_match = 1.0 if gold_ans_text in model_last_hop_text else 0.0
#             step_matches.append(last_match)

#         # Intermediate steps (middle reasoning)
#         if len(gold_hops) > 2 and len(model_steps) > 2:
#             num_intermediate_to_compare = min(len(gold_hops) - 2, len(model_steps) - 2)
            
#             for i in range(num_intermediate_to_compare):
#                 gold_mid_idx = i + 1
#                 model_mid_idx = i + 1
                
#                 gold_mid = gold_hops[gold_mid_idx].lower()
#                 model_mid = model_steps[model_mid_idx].lower()
                
#                 gold_words = set(re.findall(r'\w+', gold_mid))
#                 model_words = set(re.findall(r'\w+', model_mid))
                
#                 if gold_words and model_words:
#                     common_words = gold_words & model_words
#                     similarity = len(common_words) / max(len(gold_words), len(model_words))
#                     middle_matches.append(similarity)
#                     step_matches.append(similarity)
#                 elif gold_words or model_words:
#                     middle_matches.append(0.0)
#                     step_matches.append(0.0)

#         all_step_matches.extend(step_matches)
        
#                 if step_matches:
#             is_complete_match = all(match >= 0.6 for match in step_matches)
#             is_partial_match = False
#             if not is_complete_match:
#                 avg_middle_match = sum(middle_matches) / len(middle_matches) if middle_matches else 1.0
#                 if first_match >= 0.7 and last_match >= 0.7 and avg_middle_match >= 0.4:
#                     is_partial_match = True
            
#             if is_complete_match:
#                 complete_path_matches += 1
#                 hop_category_results[hop_cat]['matches'] += 1
#             elif is_partial_match:
#                 partial_path_matches += 1
#                 hop_category_results[hop_cat]['partial_matches'] += 1

#     cot_em_overall = complete_path_matches / total_questions if total_questions > 0 else 0
#     cot_partial_overall = (complete_path_matches + partial_path_matches) / total_questions if total_questions > 0 else 0
    
#     cot_em_by_category = {}
#     cot_partial_by_category = {}

#     for hop_cat_key in sorted_categories:
#         data = hop_category_results[hop_cat_key]
#         cat_total = data['total']
#         cot_em = data['matches'] / cat_total if cat_total > 0 else 0
#         cot_partial = (data['matches'] + data['partial_matches']) / cat_total if cat_total > 0 else 0
#         cot_em_by_category[str(hop_cat_key)] = cot_em
#         cot_partial_by_category[str(hop_cat_key)] = cot_partial
        
#     results = {
#         'CoT-EM': cot_em_overall,
#         'CoT-Partial': cot_partial_overall,
#         'Hop-Match-Rate': np.mean(all_step_matches) if all_step_matches else 0,
#         'CoT-EM by Category': cot_em_by_category,
#         'CoT-Partial by Category': cot_partial_by_category,
#         'Total Questions Evaluated for Step Verification': total_questions,
#         'Complete Path Matches': complete_path_matches,
#         'Partial Path Matches': partial_path_matches
#     }
#     return results

# def compute_cot_hop_em(df):
#     """Strict CoT EM - exact hop sequence matching"""
#     exact_matches = 0
#     total = 0
#     for _, row in df.iterrows():
#         gold_hops = row.get('gold_hops', [])
#         pred_hops = row.get('model_reasoning_steps', [])
        
#         if not gold_hops or not pred_hops or not pred_hops[0]:
#             continue
            
#         total += 1
#         if len(pred_hops) == len(gold_hops) and \
#            all(g.strip().lower() == p.strip().lower() for g, p in zip(gold_hops, pred_hops)):
#             exact_matches += 1
            
#     return exact_matches / total if total > 0 else 0.0

# def compute_cot_hop_em_by_category(df):
#     """Compute strict CoT-EM by hop category"""
#     results = {}
#     sorted_categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
    
#     for cat in sorted_categories:
#         subdf = df[df['hop_category'] == cat]
#         if subdf.empty:
#             results[str(cat)] = 0.0
#             continue
            
#         exact_matches = 0
#         total = 0
#         for _, row in subdf.iterrows():
#             gold_hops = row.get('gold_hops', [])
#             pred_hops = row.get('model_reasoning_steps', [])
#             if not gold_hops or not pred_hops or not pred_hops[0]: 
#                 continue
#             total += 1
#             if len(pred_hops) == len(gold_hops) and \
#                all(g.strip().lower() == p.strip().lower() for g, p in zip(gold_hops, pred_hops)):
#                 exact_matches += 1
                
#         results[str(cat)] = exact_matches / total if total > 0 else 0.0
#     return results

# # --- ENHANCED VISUALIZATION ---
# def visualize_hop_wise_results(hop_results, step_results, strict_cot_em_overall, cot_em_per_cat_strict, save_path=None):
#     """Enhanced visualization with better styling"""
#     plt.style.use('default')
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
#     # Main performance chart
#     categories = [cat for cat in hop_results.keys() if cat != 'overall']
#     counts = [hop_results[cat]['count'] for cat in categories]
#     em_scores = [hop_results[cat]['EM'] for cat in categories]
#     f1_scores = [hop_results[cat]['F1'] for cat in categories]
#     bleu_scores = [hop_results[cat]['BLEU'] for cat in categories]
    
#     soft_cot_em_by_cat = step_results.get('CoT-EM by Category', {})
#     soft_cot_em_scores = [soft_cot_em_by_cat.get(cat, 0) for cat in categories]
#     strict_cot_em_scores = [cot_em_per_cat_strict.get(cat, 0) for cat in categories]
    
#     x = np.arange(len(categories))
#     width = 0.15
    
#     ax1.bar(x - width*2, em_scores, width, label='Exact Match', color='#3498db', alpha=0.8)
#     ax1.bar(x - width, f1_scores, width, label='F1 Score', color='#2ecc71', alpha=0.8)
#     ax1.bar(x, bleu_scores, width, label='BLEU Score', color='#f39c12', alpha=0.8)
#     ax1.bar(x + width, soft_cot_em_scores, width, label='Soft CoT-EM', color='#e74c3c', alpha=0.8)
#     ax1.bar(x + width*2, strict_cot_em_scores, width, label='Strict CoT-EM', color='#8e44ad', alpha=0.8)
    
#     ax1.set_xlabel('Reasoning Complexity', fontweight='bold')
#     ax1.set_ylabel('Score', fontweight='bold')
#     ax1.set_title('Qwen3-0.6B Performance by Complexity', fontweight='bold', pad=20)
#     ax1.set_xticks(x)
#     ax1.set_xticklabels(categories)
#     ax1.set_ylim(0, 1.05)
#     ax1.legend(loc='upper right')
#     ax1.grid(axis='y', alpha=0.3)
    
#     # Add sample counts
#     for i, count in enumerate(counts):
#         if count > 0:
#             ax1.text(i, 0.02, f'n={count}', ha='center', va='bottom', fontsize=9,
#                     bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.2'))
    
#     # Distribution pie chart
#     if counts and sum(counts) > 0:
#         ax2.pie(counts, labels=categories, autopct='%1.1f%%', startangle=90)
#         ax2.set_title('Question Distribution by Complexity', fontweight='bold')
#     else:
#         ax2.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=ax2.transAxes)
#         ax2.set_title('No Distribution Data', fontweight='bold')
    
#     # Overall metrics text
#     overall = hop_results['overall']
#     overall_text = (f"Overall Results (n={overall['count']}):\n"
#                    f"EM: {overall['EM']:.3f} | F1: {overall['F1']:.3f} | BLEU: {overall['BLEU']:.3f}\n"
#                    f"Soft CoT-EM: {step_results['CoT-EM']:.3f} | Strict CoT-EM: {strict_cot_em_overall:.3f}")
    
#     fig.text(0.5, 0.02, overall_text, ha='center', fontsize=11, fontweight='bold',
#              bbox=dict(facecolor='lightgray', alpha=0.8, boxstyle='round,pad=0.5'))
    
#     plt.tight_layout(rect=[0, 0.1, 1, 0.96])
    
#     # Save plot
#     if save_path:
#         plt.savefig(save_path, dpi=300, bbox_inches='tight')
#         print(f"Visualization saved to {save_path}")
#     plt.show()

# # --- MAIN EXECUTION ---
# def main():
#     """Main execution function"""
#     print("=== MenatQA Evaluation Pipeline ===\n")
    
#     # Configuration
#     OUTPUT_DIR = Path.cwd() / "menatqa_evaluation"
#     OUTPUT_DIR.mkdir(exist_ok=True)
    
#     DATASET_FILE = OUTPUT_DIR / 'MenatQA.json'
#     MODEL_NAME = "Qwen/Qwen3-0.6B"
#     OUTPUT_IMAGE_PATH = OUTPUT_DIR / 'menatqa_qwen3_performance.png'
#     OUTPUT_RESULTS_PATH = OUTPUT_DIR / 'menatqa_qwen3_results.json'
    
#     print(f"Output directory: {OUTPUT_DIR}")
#     print(f"Model: {MODEL_NAME}\n")
    
#     # Load and preprocess data
#     print("📊 Loading dataset...")
#     data = load_menatqa_dataset(DATASET_FILE)
    
#     print("🔄 Preprocessing dataset...")
#     df = preprocess_dataset(data)
    
#     # Run model inference
#     print(f"\n🤖 Running {MODEL_NAME} inference...")
#     df_results = run_qwen_predictions(df, model_name=MODEL_NAME)
    
#     # Evaluate results
#     print("\n📈 Evaluating results...")
#     hop_results = evaluate_hop_wise_accuracy(df_results)
#     step_verification_results = evaluate_step_verification(df_results)
#     strict_cot_em_overall = compute_cot_hop_em(df_results)
#     strict_cot_em_by_category = compute_cot_hop_em_by_category(df_results)
    
#     # Display results
#     print("\n" + "="*60)
#     print("📊 RESULTS SUMMARY")
#     print("="*60)
    
#     print("\n--- Answer Accuracy by Complexity ---")
#     for category, metrics in hop_results.items():
#         if category == 'overall': continue
#         print(f"{category:>10} (n={metrics['count']:>3}): EM={metrics['EM']:.3f}, F1={metrics['F1']:.3f}, BLEU={metrics['BLEU']:.3f}")
    
#     overall = hop_results['overall']
#     print(f"{'Overall':>10} (n={overall['count']:>3}): EM={overall['EM']:.3f}, F1={overall['F1']:.3f}, BLEU={overall['BLEU']:.3f}")
    
#     print("\n--- Reasoning Path Verification ---")
#     print(f"Soft CoT-EM (Path):   {step_verification_results['CoT-EM']:.3f}")
#     print(f"CoT-Partial (Path):   {step_verification_results['CoT-Partial']:.3f}")
#     print(f"Hop-Match-Rate:       {step_verification_results['Hop-Match-Rate']:.3f}")
#     print(f"Strict CoT-EM (Path): {strict_cot_em_overall:.3f}")
    
#     print("\nBy Category - Soft CoT-EM:")
#     for cat, score in step_verification_results['CoT-EM by Category'].items():
#         print(f"  {cat}: {score:.3f}")
    
#     # Generate visualization
#     print("\n📊 Generating visualization...")
#     visualize_hop_wise_results(
#         hop_results, step_verification_results, 
#         strict_cot_em_overall, strict_cot_em_by_category, 
#         save_path=OUTPUT_IMAGE_PATH
#     )
    
#     # Save comprehensive results
#     summary_results = {
#         'model_name': MODEL_NAME,
#         'dataset_info': {
#             'name': 'MenatQA',
#             'total_questions_processed': len(df_results),
#             'hop_distribution': df_results['hop_category'].value_counts().to_dict()
#         },
#         'answer_accuracy_metrics': hop_results,
#         'reasoning_path_verification_metrics': step_verification_results,
#         'strict_cot_em_overall': float(strict_cot_em_overall),
#         'strict_cot_em_by_category': {k: float(v) for k, v in strict_cot_em_by_category.items()}
#     }
    
#     with open(OUTPUT_RESULTS_PATH, 'w') as f:
#         json.dump(summary_results, f, indent=2, default=str)
    
#     print(f"\n✅ Evaluation complete!")
#     print(f"📁 Results saved to: {OUTPUT_RESULTS_PATH}")
#     print(f"📊 Visualization saved to: {OUTPUT_IMAGE_PATH}")
    
#     return df_results, summary_results

# if __name__ == '__main__':
#     print("🚀 Running full MenatQA evaluation on Qwen3-0.6B...")
#     main()

In [ ]:
!pip install accelerate

In [ ]:
import json
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm
import torch
from pathlib import Path

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Download NLTK data
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

# --- DATA ---
def load_menatqa_dataset(file_path='./MenatQA.json'):
    file_path = Path(file_path)
    
    if not file_path.exists():
        import urllib.request
        print(f"Downloading MenatQA to {file_path}...")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/weiyifan1023/MenatQA/main/datasets/MenatQA.json",
            str(file_path)
        )
    
    with open(file_path, 'r') as f:
        data = json.load(f)
    print(f"Loaded MenatQA dataset with {len(data)} examples")
    return data

def extract_reasoning_hops(example):
    question = example.get('question', '')
    answer = example.get('answer', '')
    q_type = example.get('type', '')
    time_scope = example.get('time_scope', '')
    
    sentences = [s.strip() for s in question.split('.') if s.strip()]
    clauses = len([c for c in re.split(r'and|or|but|because|when|if', question.lower()) if c.strip()])
    capitalized_words = len([w for w in question.split() if w and w[0].isupper()])
    
    complexity_score = 1
    complexity_score += min(1, len(sentences) - 1)
    complexity_score += min(1, (clauses - 1) // 2)
    complexity_score += min(1, capitalized_words // 3)
    if time_scope:
        complexity_score += 1
    complexity_score = min(4, max(1, complexity_score))
    
    hops = []
    hops.append(f"Understand the question: {question}")
    if complexity_score >= 2:
        if time_scope:
            hops.append(f"Identify time context: {time_scope}")
        else:
            hops.append(f"Recognize question type: {q_type}")
    if complexity_score >= 3:
        hops.append("Retrieve relevant facts and information")
    if complexity_score >= 4:
        hops.append("Analyze relationships between facts")
    hops.append(f"Formulate answer: {answer}")
    
    return hops, complexity_score

def preprocess_dataset(data):
    processed_data = []
    hop_counts = {1: 0, 2: 0, 3: 0, 4: 0}
    
    for item in data:
        gold_hops, complexity_score = extract_reasoning_hops(item)
        hop_counts[min(4, complexity_score)] += 1
        
        entry = {
            'ID': item.get('ID', ''),
            'question': item.get('question', ''),
            'answer': item.get('answer', ''),
            'type': item.get('type', ''),
            'time_scope': item.get('time_scope', ''),
            'gold_hops': gold_hops,
            'hop_count': complexity_score,
            'model_prediction': '',
            'model_reasoning_steps': []
        }
        processed_data.append(entry)
    
    df = pd.DataFrame(processed_data)
    df['hop_category'] = pd.cut(
        df['hop_count'],
        bins=[-1, 1, 2, 3, float('inf')],
        labels=['1-hop', '2-hop', '3-hop', '4+-hop']
    )
    
    print(f"Dataset preprocessed: {len(df)} questions")
    for i in range(1, 5):
        print(f"{i}-hop questions: {hop_counts[i]} ({hop_counts[i]/len(df):.1%})")
    
    return df

# --- MODEL ---
def run_qwen_predictions(df, model_name="Qwen/Qwen2-0.5B"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading {model_name} on {device}...")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
    
    pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    device=0 if device == 'cuda' else -1,
    max_new_tokens=400,  # Increased from 256
    temperature=0.3,     # Increased from 0.01
    do_sample=True,
    top_p=0.9,          # Added for better generation
    repetition_penalty=1.1  # Added to avoid repetition
)

    def format_cot_prompt(question):
        return f"""You are an expert at answering questions step by step. Follow this format exactly:

Step 1: [Understand what the question is asking]
Step 2: [Identify key information needed]
Step 3: [Reason through the problem]
Final Answer: [Give the direct answer]

Question: {question}

Let me solve this step by step:"""

    all_preds, all_steps = [], []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating predictions"):
        prompt = format_cot_prompt(row["question"])
        generation = pipe(prompt)[0]['generated_text']
        output = generation[len(prompt):] if generation.startswith(prompt) else generation
        
        # Better parsing
        lines = [x.strip() for x in output.split('\n') if x.strip()]
        steps = []
        model_answer = ""
        
        for line in lines:
            if line.lower().startswith('final answer:') or line.lower().startswith('answer:'):
                model_answer = line.split(':', 1)[-1].strip()
                steps.append(line)
            elif line.lower().startswith('step') or line.lower().startswith('1.') or line.lower().startswith('2.'):
                steps.append(line)
            elif not model_answer and line:  # If no explicit answer found, use last meaningful line
                model_answer = line
                
        if not steps:
            steps = [output.strip()] if output.strip() else ["No reasoning provided"]
        if not model_answer:
            model_answer = steps[-1] if steps else "No answer"
            
        all_preds.append(model_answer)
        all_steps.append(steps)
    
    df = df.copy()
    df['model_prediction'] = all_preds
    df['model_reasoning_steps'] = all_steps
    return df

# --- METRICS ---
def compute_exact_match(prediction, gold):
    pred_clean = re.sub(r'[^\w\s]', '', str(prediction).lower().strip())
    gold_clean = re.sub(r'[^\w\s]', '', str(gold).lower().strip())
    
    # Direct match
    if pred_clean == gold_clean:
        return 1
    
    # Check if gold is contained in prediction
    if gold_clean in pred_clean:
        return 1
        
    # Check if prediction is contained in gold
    if pred_clean in gold_clean:
        return 1
        
    # Token-level similarity for partial credit
    pred_tokens = set(pred_clean.split())
    gold_tokens = set(gold_clean.split())
    if pred_tokens and gold_tokens:
        jaccard = len(pred_tokens & gold_tokens) / len(pred_tokens | gold_tokens)
        return 1 if jaccard > 0.8 else 0
    
    return 0

def compute_f1(prediction, gold):
    pred_tokens = str(prediction).lower().split()
    gold_tokens = str(gold).lower().split()
    common = set(pred_tokens) & set(gold_tokens)
    if not pred_tokens or not gold_tokens:
        return 0.0
    prec = len(common) / len(pred_tokens)
    rec = len(common) / len(gold_tokens)
    if prec + rec == 0:
        return 0.0
    return 2 * prec * rec / (prec + rec)

def compute_bleu(prediction, gold):
    pred_tokens = str(prediction).lower().split()
    gold_tokens = str(gold).lower().split()
    if not pred_tokens or not gold_tokens:
        return 0.0
    reference = [gold_tokens]
    candidate = pred_tokens
    weights = (0.25, 0.25, 0.25, 0.25)
    chencherry = SmoothingFunction()
    try:
        bleu_score = sentence_bleu(reference, candidate, weights=weights, smoothing_function=chencherry.method1)
    except:
        bleu_score = 0.0
    return bleu_score

def evaluate_hop_wise_accuracy(df):
    categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
    hop_results = {}
    overall_em, overall_f1, overall_bleu, overall_count = 0, 0, 0, 0
    
    for cat in categories:
        subdf = df[df['hop_category'] == cat]
        if subdf.empty:
            hop_results[str(cat)] = {'count': 0, 'EM': 0, 'F1': 0, 'BLEU': 0}
            continue

        ems = [compute_exact_match(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
        f1s = [compute_f1(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
        bleus = [compute_bleu(row['model_prediction'], row['answer']) for _, row in subdf.iterrows()]
        
        count = len(subdf)
        mean_em = np.mean(ems) if ems else 0
        mean_f1 = np.mean(f1s) if f1s else 0
        mean_bleu = np.mean(bleus) if bleus else 0
        
        hop_results[str(cat)] = {'count': count, 'EM': mean_em, 'F1': mean_f1, 'BLEU': mean_bleu}
        
        overall_em += sum(ems)
        overall_f1 += sum(f1s)
        overall_bleu += sum(bleus)
        overall_count += count
        
    hop_results['overall'] = {
        'count': overall_count,
        'EM': overall_em / overall_count if overall_count else 0,
        'F1': overall_f1 / overall_count if overall_count else 0,
        'BLEU': overall_bleu / overall_count if overall_count else 0,
    }
    return hop_results

def evaluate_step_verification(df):
    all_step_matches = []
    complete_path_matches = 0
    partial_path_matches = 0
    total_questions = 0
    sorted_categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
    hop_category_results = defaultdict(lambda: {'matches': 0, 'partial_matches': 0, 'total': 0})

    for _, row in df.iterrows():
        gold_hops = row['gold_hops']
        model_steps = row['model_reasoning_steps']
        
        if not gold_hops or not model_steps or not model_steps[0]:
            continue
            
        total_questions += 1
        hop_cat = row['hop_category']
        if hop_cat is None: continue

        hop_category_results[hop_cat]['total'] += 1
        
        step_matches = []
        first_match = 0
        last_match = 0
        middle_matches = []

        # First step
        gold_q_terms = set(re.findall(r'\w+', gold_hops[0].lower()))
        model_q_terms = set(re.findall(r'\w+', model_steps[0].lower()))
        if gold_q_terms:
            common_terms = gold_q_terms & model_q_terms
            first_match = len(common_terms) / len(gold_q_terms)
        step_matches.append(first_match)

        # Last step
        if len(gold_hops) > 1 and len(model_steps) > 1:
            gold_ans_text = row['answer'].lower()
            model_last_hop_text = model_steps[-1].lower()
            last_match = 1.0 if gold_ans_text in model_last_hop_text else 0.0
            step_matches.append(last_match)

        # Intermediate steps
        if len(gold_hops) > 2 and len(model_steps) > 2:
            num_intermediate_to_compare = min(len(gold_hops) - 2, len(model_steps) - 2)
            for i in range(num_intermediate_to_compare):
                gold_mid_idx = i + 1
                model_mid_idx = i + 1
                
                gold_mid = gold_hops[gold_mid_idx].lower()
                model_mid = model_steps[model_mid_idx].lower()
                
                gold_words = set(re.findall(r'\w+', gold_mid))
                model_words = set(re.findall(r'\w+', model_mid))
                
                if gold_words and model_words:
                    common_words = gold_words & model_words
                    similarity = len(common_words) / max(len(gold_words), len(model_words))
                    middle_matches.append(similarity)
                    step_matches.append(similarity)
                elif gold_words or model_words:
                    middle_matches.append(0.0)
                    step_matches.append(0.0)

        all_step_matches.extend(step_matches)
        
        if step_matches:
            is_complete_match = all(match >= 0.6 for match in step_matches)
            is_partial_match = False
            if not is_complete_match:
                avg_middle_match = sum(middle_matches) / len(middle_matches) if middle_matches else 1.0
                if first_match >= 0.7 and last_match >= 0.7 and avg_middle_match >= 0.4:
                    is_partial_match = True
            
            if is_complete_match:
                complete_path_matches += 1
                hop_category_results[hop_cat]['matches'] += 1
            elif is_partial_match:
                partial_path_matches += 1
                hop_category_results[hop_cat]['partial_matches'] += 1

    cot_em_overall = complete_path_matches / total_questions if total_questions > 0 else 0
    cot_partial_overall = (complete_path_matches + partial_path_matches) / total_questions if total_questions > 0 else 0
    
    cot_em_by_category = {}
    cot_partial_by_category = {}

    for hop_cat_key in sorted_categories:
        data = hop_category_results[hop_cat_key]
        cat_total = data['total']
        cot_em = data['matches'] / cat_total if cat_total > 0 else 0
        cot_partial = (data['matches'] + data['partial_matches']) / cat_total if cat_total > 0 else 0
        cot_em_by_category[str(hop_cat_key)] = cot_em
        cot_partial_by_category[str(hop_cat_key)] = cot_partial
        
    results = {
        'CoT-EM': cot_em_overall,
        'CoT-Partial': cot_partial_overall,
        'Hop-Match-Rate': np.mean(all_step_matches) if all_step_matches else 0,
        'CoT-EM by Category': cot_em_by_category,
        'CoT-Partial by Category': cot_partial_by_category,
        'Total Questions Evaluated for Step Verification': total_questions
    }
    return results

def compute_cot_hop_em(df):
    exact_matches = 0
    total = 0
    for _, row in df.iterrows():
        gold_hops = row.get('gold_hops', [])
        pred_hops = row.get('model_reasoning_steps', [])
        
        if not gold_hops or not pred_hops or not pred_hops[0]:
            continue
            
        total += 1
        if len(pred_hops) == len(gold_hops) and \
           all(g.strip().lower() == p.strip().lower() for g, p in zip(gold_hops, pred_hops)):
            exact_matches += 1
            
    return exact_matches / total if total > 0 else 0.0

def compute_cot_hop_em_by_category(df):
    results = {}
    sorted_categories = sorted([cat for cat in df['hop_category'].unique() if cat is not None], key=lambda x: x[0])
    
    for cat in sorted_categories:
        subdf = df[df['hop_category'] == cat]
        if subdf.empty:
            results[str(cat)] = 0.0
            continue
            
        exact_matches = 0
        total = 0
        for _, row in subdf.iterrows():
            gold_hops = row.get('gold_hops', [])
            pred_hops = row.get('model_reasoning_steps', [])
            if not gold_hops or not pred_hops or not pred_hops[0]: 
                continue
            total += 1
            if len(pred_hops) == len(gold_hops) and \
               all(g.strip().lower() == p.strip().lower() for g, p in zip(gold_hops, pred_hops)):
                exact_matches += 1
                
        results[str(cat)] = exact_matches / total if total > 0 else 0.0
    return results

# --- VISUALIZATION ---
def visualize_hop_wise_results(hop_results, step_results, strict_cot_em_overall, cot_em_per_cat_strict, save_path=None):
    plt.figure(figsize=(15, 10))
    
    categories = [cat for cat in hop_results.keys() if cat != 'overall']
    counts = [hop_results[cat]['count'] for cat in categories]
    em_scores = [hop_results[cat]['EM'] for cat in categories]
    f1_scores = [hop_results[cat]['F1'] for cat in categories]
    bleu_scores = [hop_results[cat]['BLEU'] for cat in categories]
    
    soft_cot_em_by_cat = step_results.get('CoT-EM by Category', {})
    soft_cot_em_scores = [soft_cot_em_by_cat.get(cat, 0) for cat in categories]
    strict_cot_em_scores = [cot_em_per_cat_strict.get(cat, 0) for cat in categories]
    
    x = np.arange(len(categories))
    width = 0.15
    
    plt.bar(x - width*2, em_scores, width, label='Exact Match', color='#3498db', alpha=0.8)
    plt.bar(x - width, f1_scores, width, label='F1 Score', color='#2ecc71', alpha=0.8)
    plt.bar(x, bleu_scores, width, label='BLEU Score', color='#f39c12', alpha=0.8)
    plt.bar(x + width, soft_cot_em_scores, width, label='Soft CoT-EM', color='#e74c3c', alpha=0.8)
    plt.bar(x + width*2, strict_cot_em_scores, width, label='Strict CoT-EM', color='#8e44ad', alpha=0.8)
    
    plt.xlabel('Reasoning Complexity', fontweight='bold')
    plt.ylabel('Score', fontweight='bold')
    plt.title('Qwen Performance on MenatQA by Reasoning Complexity', fontweight='bold', pad=20)
    plt.xticks(x, categories)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    for i, count in enumerate(counts):
        if count > 0:
            plt.text(i, 0.02, f'n={count}', ha='center', va='bottom', fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.2'))
    
    overall = hop_results['overall']
    overall_text = (f"Overall Results (n={overall['count']}):\n"
                   f"EM: {overall['EM']:.3f} | F1: {overall['F1']:.3f} | BLEU: {overall['BLEU']:.3f}\n"
                   f"Soft CoT-EM: {step_results['CoT-EM']:.3f} | Strict CoT-EM: {strict_cot_em_overall:.3f}")
    
    plt.figtext(0.5, 0.02, overall_text, ha='center', fontsize=11, fontweight='bold',
                bbox=dict(facecolor='lightgray', alpha=0.8, boxstyle='round,pad=0.5'))
    
    plt.tight_layout(rect=[0, 0.1, 1, 0.96])
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Visualization saved to {save_path}")
    plt.show()

# --- MAIN ---
if __name__ == '__main__':
    # Configuration
    DATASET_FILE = './MenatQA.json'
    MODEL_NAME = "Qwen/Qwen3-0.6B"
    OUTPUT_IMAGE_PATH = './menatqa_qwen_performance.png'
    OUTPUT_RESULTS_PATH = './menatqa_qwen_results.json'

    print("=== MenatQA Evaluation on Qwen ===\n")
    
    # Load data
    print("Loading MenatQA dataset...")
    data = load_menatqa_dataset(DATASET_FILE)
    
    # Preprocess
    print("Preprocessing dataset...")
    df = preprocess_dataset(data)
    
    # Run model
    print(f"Running {MODEL_NAME} on {len(df)} questions...")
    df_results = run_qwen_predictions(df, model_name=MODEL_NAME)
    
    # Evaluate
    print("Evaluating results...")
    hop_results = evaluate_hop_wise_accuracy(df_results)
    step_verification_results = evaluate_step_verification(df_results)
    strict_cot_em_overall = compute_cot_hop_em(df_results)
    strict_cot_em_by_category = compute_cot_hop_em_by_category(df_results)
    
    # Print results
    print("\n" + "="*60)
    print("RESULTS SUMMARY")
    print("="*60)
    
    print("\n--- Answer Accuracy by Complexity ---")
    for category, metrics in hop_results.items():
        if category == 'overall': continue
        print(f"{category}: EM={metrics['EM']:.3f}, F1={metrics['F1']:.3f}, BLEU={metrics['BLEU']:.3f} (n={metrics['count']})")
    
    overall = hop_results['overall']
    print(f"Overall: EM={overall['EM']:.3f}, F1={overall['F1']:.3f}, BLEU={overall['BLEU']:.3f} (n={overall['count']})")
    
    print("\n--- Reasoning Path Verification ---")
    print(f"Soft CoT-EM: {step_verification_results['CoT-EM']:.3f}")
    print(f"CoT-Partial: {step_verification_results['CoT-Partial']:.3f}")
    print(f"Hop-Match-Rate: {step_verification_results['Hop-Match-Rate']:.3f}")
    print(f"Strict CoT-EM: {strict_cot_em_overall:.3f}")
    
    # Visualize
    print("\nGenerating visualization...")
    visualize_hop_wise_results(
        hop_results, 
        step_verification_results, 
        strict_cot_em_overall, 
        strict_cot_em_by_category, 
        save_path=OUTPUT_IMAGE_PATH
    )
    
    # Save results
    summary_results = {
        'model_name': MODEL_NAME,
        'dataset_info': {
            'name': 'MenatQA',
            'total_questions_processed': len(df_results),
            'hop_distribution': df_results['hop_category'].value_counts().to_dict()
        },
        'answer_accuracy_metrics': hop_results,
        'reasoning_path_verification_metrics': step_verification_results,
        'strict_cot_em_overall': float(strict_cot_em_overall),
        'strict_cot_em_by_category': {k: float(v) for k, v in strict_cot_em_by_category.items()}
    }
    
    with open(OUTPUT_RESULTS_PATH, 'w') as f:
        json.dump(summary_results, f, indent=2)
    
    print(f"\nEvaluation complete!")
    print(f"Results saved to: {OUTPUT_RESULTS_PATH}")
    print(f"Visualization saved to: {OUTPUT_IMAGE_PATH}")

=== MenatQA Evaluation on Qwen ===

Loading MenatQA dataset...
Loaded MenatQA dataset with 999 examples
Preprocessing dataset...
Dataset preprocessed: 999 questions
1-hop questions: 0 (0.0%)
2-hop questions: 135 (13.5%)
3-hop questions: 698 (69.9%)
4-hop questions: 166 (16.6%)
Running Qwen/Qwen3-0.6B on 999 questions...
Loading Qwen/Qwen3-0.6B on cuda...


Device set to use cuda:0
Generating predictions:  11%|█         | 106/999 [21:59<3:04:04, 12.37s/it]